In [1]:
from pathlib import Path
import json
import pandas as pd
import re

In [2]:
folder_ignore = '.ipynb_checkpoints'
path_list =[  i for i  in Path.cwd().rglob('*.json') if folder_ignore not in list(i.parent.parts) ]

In [3]:
path_list

[WindowsPath('C:/Users/dmitr/project_loc/MainTopicsExtractor/main_topics_cloud/reserch/data/example/result.json'),
 WindowsPath('C:/Users/dmitr/project_loc/MainTopicsExtractor/main_topics_cloud/reserch/data/дата/result.json'),
 WindowsPath('C:/Users/dmitr/project_loc/MainTopicsExtractor/main_topics_cloud/reserch/data/маркетинг/result.json'),
 WindowsPath('C:/Users/dmitr/project_loc/MainTopicsExtractor/main_topics_cloud/reserch/data/менеджмент/result.json')]

In [4]:
#  очищает стоку, оставляя только буквы, цифры, запятые, точки, восклицательные и вопросительные знаки, а также пробелы.
pattern  = r'[^\w,.\!? ]'
def cleaned_string(string, pattern=pattern):
    return re.sub(pattern, '', string).strip()

In [5]:
# Загрузка json
def load_json(path: Path) -> dict:
    with open(path,'r',encoding='utf-8') as file:
        return json.load(file)

In [6]:
# парсит словарь,  если значение ключа коллекция выводит тип данных и размер
def parsing_dict( dt: dict) -> None:
    for key in dt.keys():
        if  not isinstance(dt.get(key),list):
             print(f'key: {key}: {dt.get(key)}')   
        else: print(f'key: {key}: {type(dt.get(key))}, len: {len(dt.get(key))}')

In [7]:
# парсит индекс messages с наибольшим  количеством ключей + True - во всех сообщениях одинаковое количество ключей
def index_max_key(dt):
    keys_messages,items  =[],[]
    for i in range(len (dt['messages'])):
        keys_messages.append(tuple(dt['messages'][i].keys()))              

    print(set(keys_messages) == len(keys_messages))
    for item in set(keys_messages) :  
        items.append(item)
    index_max_keys = keys_messages.index(max(set(keys_messages), key=len))
    return index_max_keys    

In [8]:
#  создает  дата фрейм из словаря
def build_df_chat( dict_to_df : dict, columns: dict[str,str]) -> pd.DataFrame:
    df_dict = {
    **{columns.get(col): [dict_to_df['messages'][x].get(col) for x in range(len(dict_to_df['messages']))]
        for col in columns
    },
    'text': [ ' '.join( i.get('text') for  i in dict_to_df['messages'][x]['text_entities'])
        for x in range( len(dict_to_df['messages']))
    ]}    
    df = pd.DataFrame(df_dict)
    df['date'] =  pd.to_datetime(df['date'])
    if 'user_name_actor' in df.columns:
        df['user_name'] = df['user_name'].fillna(df['user_name_actor'])
        del df['user_name_actor']
    df['user_name'] = df['user_name'].fillna('')
    df['chat_name'] =dict_to_df.get('name')
    df['chat_id'] =dict_to_df.get('id')
    column =list(df.columns)
    df = df[column[:1]+ column[-2:]+column[1:-2]]
  
    if 'user_name' in df.columns:
        df['first_name'] = df['user_name'].apply( lambda x:  x.split()[0] if len( x.split())>0 else '')
        df['last_name'] = df['user_name'].apply( lambda x: x.split()[1] if len( x.split())>1 else '' )
        column =list(df.columns)
        df = df[column[:-3]+column[-2:]+[column[-3]]]

    if 'actor_id' in df.columns:
        df['from_id'] = df['from_id'].fillna(df['actor_id'])
        del df['actor_id']
        # del df['user_name_actor']

    if 'from_id' in df.columns:
        df['sender_id'] = df['from_id'].str.extract(r'(\d+)')
        df['sender_type'] = df['from_id'].str.replace(r'\d+', '', regex=True)
        del df['from_id']
        column =list(df.columns)
        df=df[column[:3]+column[-2:]+column[3:-2]]
        
    # df = df.fillna(0)
    return df
    

### 1. Парсинг result.json (дата)

In [9]:
path_list

[WindowsPath('C:/Users/dmitr/project_loc/MainTopicsExtractor/main_topics_cloud/reserch/data/example/result.json'),
 WindowsPath('C:/Users/dmitr/project_loc/MainTopicsExtractor/main_topics_cloud/reserch/data/дата/result.json'),
 WindowsPath('C:/Users/dmitr/project_loc/MainTopicsExtractor/main_topics_cloud/reserch/data/маркетинг/result.json'),
 WindowsPath('C:/Users/dmitr/project_loc/MainTopicsExtractor/main_topics_cloud/reserch/data/менеджмент/result.json')]

In [10]:
result_dict_1 = load_json(path_list[1])

In [11]:
parsing_dict(result_dict_1)

key: name: 💬 Data Practicum Chat
key: type: private_supergroup
key: id: 1379846874
key: messages: <class 'list'>, len: 1007


In [12]:
index_max_keys  = index_max_key(result_dict_1)

False


In [13]:
parsing_dict(result_dict_1['messages'][index_max_keys])

key: id: 266871
key: type: message
key: date: 2025-02-04T16:29:08
key: date_unixtime: 1738675748
key: edited: 2025-02-04T16:30:12
key: edited_unixtime: 1738675812
key: from: Мариян Умагалова
key: from_id: user168336377
key: reply_to_message_id: 266834
key: file: (File not included. Change data exporting settings to download.)
key: file_name: sticker.webm
key: file_size: 90810
key: thumbnail: (File not included. Change data exporting settings to download.)
key: thumbnail_file_size: 9936
key: media_type: sticker
key: sticker_emoji: 😒
key: mime_type: video/webm
key: duration_seconds: 3
key: width: 512
key: height: 488
key: text: 
key: text_entities: <class 'list'>, len: 0
key: reactions: <class 'list'>, len: 1


In [14]:
set(result_dict_1['messages'][i]['type'] for i in range( len(result_dict_1['messages'])))

{'message', 'service'}

In [15]:
parsing_dict(result_dict_1['messages'][min([i for i in range( len(result_dict_1['messages'])) if result_dict_1['messages'][i]['type'] =='service'])])

key: id: 266690
key: type: service
key: date: 2025-02-01T22:40:19
key: date_unixtime: 1738438819
key: actor: Елизавета
key: actor_id: user200103497
key: action: join_group_by_link
key: inviter: Group
key: text: 
key: text_entities: <class 'list'>, len: 0


In [16]:
# словарь ключ - ключ из словаря  messages  значение соответствующее название столбца  df   
columns ={'date': 'date', 'id':'message_id','type':'type', 'from_id': 'from_id', 'actor_id':'actor_id' , 'reply_to_message_id': 'reply_to_message_id',
         'from': 'user_name', 'actor':'user_name_actor'}

In [17]:
df_1 = build_df_chat(result_dict_1, columns)

In [18]:
df_1.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 1007 entries, 0 to 1006
Data columns (total 12 columns):
 #   Column               Non-Null Count  Dtype         
---  ------               --------------  -----         
 0   date                 1007 non-null   datetime64[ns]
 1   chat_name            1007 non-null   object        
 2   chat_id              1007 non-null   int64         
 3   sender_id            1007 non-null   object        
 4   sender_type          1007 non-null   object        
 5   message_id           1007 non-null   int64         
 6   type                 1007 non-null   object        
 7   reply_to_message_id  689 non-null    float64       
 8   user_name            1007 non-null   object        
 9   first_name           1007 non-null   object        
 10  last_name            1007 non-null   object        
 11  text                 1007 non-null   object        
dtypes: datetime64[ns](1), float64(1), int64(2), object(8)
memory usage: 94.5+ KB


In [19]:
pd.concat([df_1.head(3),df_1.tail(3)])

,date,chat_name,chat_id,sender_id,sender_type,message_id,type,reply_to_message_id,user_name,first_name,last_name,text
0,2025-02-01 22:40:19,💬 Data Practicum Chat,1379846874,200103497,user,266690,service,NaN,Елизавета,Елизавета,,
1,2025-02-02 21:59:34,💬 Data Practicum Chat,1379846874,217302209,user,266691,service,NaN,Agamet Agametov,Agamet,Agametov,
2,2025-02-03 11:28:38,💬 Data Practicum Chat,1379846874,312724902,user,266693,message,NaN,Olga Varavina,Olga,Varavina,Всем большой привет! Приглашаю на свой уютный ...
1004,2025-02-27 18:33:30,💬 Data Practicum Chat,1379846874,5459178210,user,267831,message,267830.0,Дмитрий Лукьянов,Дмитрий,Лукьянов,"Может, тебе просто пойти в лес?"
1005,2025-02-27 20:01:03,💬 Data Practicum Chat,1379846874,1763861461,channel,267832,message,NaN,DATA ALUMNI PRACTICUM,DATA,ALUMNI,Книжный клуб «Data Insights!» \n\n «Data Insig...
1006,2025-02-27 22:33:36,💬 Data Practicum Chat,1379846874,295904416,user,267833,message,267831.0,Andrey Novikov,Andrey,Novikov,"сам не можешь, нужен сопровождаюший?\n\nбез пр..."


### 2. Парсинг result.json (Мaркетинг)

In [20]:
path_list

[WindowsPath('C:/Users/dmitr/project_loc/MainTopicsExtractor/main_topics_cloud/reserch/data/example/result.json'),
 WindowsPath('C:/Users/dmitr/project_loc/MainTopicsExtractor/main_topics_cloud/reserch/data/дата/result.json'),
 WindowsPath('C:/Users/dmitr/project_loc/MainTopicsExtractor/main_topics_cloud/reserch/data/маркетинг/result.json'),
 WindowsPath('C:/Users/dmitr/project_loc/MainTopicsExtractor/main_topics_cloud/reserch/data/менеджмент/result.json')]

In [21]:
 result_dict_2 =load_json(path_list[2])

In [22]:
parsing_dict(result_dict_2)

key: name: 💬 Marketing Practicum Chat
key: type: private_supergroup
key: id: 1782474569
key: messages: <class 'list'>, len: 3680


In [23]:
index_max_keys  = index_max_key(result_dict_2)
parsing_dict(result_dict_2['messages'][index_max_keys])

False
key: id: 45608
key: type: message
key: date: 2025-02-20T10:58:54
key: date_unixtime: 1740038334
key: edited: 2025-02-20T10:59:38
key: edited_unixtime: 1740038378
key: from: Ed
key: from_id: user6615497723
key: reply_to_message_id: 45607
key: file: (File not included. Change data exporting settings to download.)
key: file_name: sticker.webm
key: file_size: 169864
key: thumbnail: (File not included. Change data exporting settings to download.)
key: thumbnail_file_size: 11180
key: media_type: sticker
key: sticker_emoji: 😌
key: mime_type: video/webm
key: duration_seconds: 3
key: width: 512
key: height: 512
key: text: 
key: text_entities: <class 'list'>, len: 0
key: reactions: <class 'list'>, len: 1


In [24]:
set(result_dict_2['messages'][i]['type'] for i in range( len(result_dict_2['messages'])))

{'message'}

In [25]:
# словарь ключ - ключ из словаря  messages  значение соответствующее название столбца  df   
columns ={'date': 'date', 'id':'message_id','type':'type', 'from_id': 'from_id', 'actor_id':'actor_id' , 'reply_to_message_id': 'reply_to_message_id',
         'from': 'user_name', 'actor':'user_name_actor'}

In [26]:
df_2 = build_df_chat(result_dict_2, columns)

In [27]:
pd.concat([df_2.head(3),df_2.tail(3)])

,date,chat_name,chat_id,sender_id,sender_type,message_id,type,reply_to_message_id,user_name,first_name,last_name,text
0,2025-02-01 07:35:54,💬 Marketing Practicum Chat,1782474569,1272493086,user,42471,message,NaN,Pasha Sannikov,Pasha,Sannikov,"Доброе утро, люди! Это ОК, в РСЯ алгоритмы выб..."
1,2025-02-01 07:40:03,💬 Marketing Practicum Chat,1782474569,1272493086,user,42472,message,NaN,Pasha Sannikov,Pasha,Sannikov,"А вот регионы, в которых живут ITшники с интер..."
2,2025-02-01 07:48:53,💬 Marketing Practicum Chat,1782474569,1272493086,user,42473,message,NaN,Pasha Sannikov,Pasha,Sannikov,"Если смотреть по интересам, то кажется, что ау..."
3677,2025-02-27 20:24:44,💬 Marketing Practicum Chat,1782474569,1912824918,user,46402,message,NaN,S,S,,Пишите оператору (позовите пожалуйста оператор...
3678,2025-02-27 20:29:44,💬 Marketing Practicum Chat,1782474569,1829741608,user,46403,message,46402.0,Алексей Ермолов,Алексей,Ермолов,Отлично! Благодарю.
3679,2025-02-27 20:30:53,💬 Marketing Practicum Chat,1782474569,1912824918,user,46405,message,46403.0,S,S,,"Да там часто отвечает бот, поэтому надо добить..."


### 3. Парсинг  result.json (Менеджмент)

In [28]:
path_list

[WindowsPath('C:/Users/dmitr/project_loc/MainTopicsExtractor/main_topics_cloud/reserch/data/example/result.json'),
 WindowsPath('C:/Users/dmitr/project_loc/MainTopicsExtractor/main_topics_cloud/reserch/data/дата/result.json'),
 WindowsPath('C:/Users/dmitr/project_loc/MainTopicsExtractor/main_topics_cloud/reserch/data/маркетинг/result.json'),
 WindowsPath('C:/Users/dmitr/project_loc/MainTopicsExtractor/main_topics_cloud/reserch/data/менеджмент/result.json')]

In [29]:
 result_dict_3 =load_json(path_list[3])

In [30]:
parsing_dict(result_dict_3)

key: name: MANAGEMENT ALUMNI PRACTICUM
key: type: private_channel
key: id: 1586396652
key: messages: <class 'list'>, len: 13


In [31]:
index_max_keys  = index_max_key(result_dict_3)
parsing_dict(result_dict_3['messages'][index_max_keys])

False
key: id: 857
key: type: message
key: date: 2025-02-02T14:07:55
key: date_unixtime: 1738494475
key: edited: 2025-02-03T13:05:36
key: edited_unixtime: 1738577136
key: from: MANAGEMENT ALUMNI PRACTICUM
key: from_id: channel1586396652
key: photo: photos/photo_368@02-02-2025_14-07-55.jpg
key: photo_file_size: 130046
key: width: 1280
key: height: 1280
key: text: <class 'list'>, len: 14
key: text_entities: <class 'list'>, len: 14
key: reactions: <class 'list'>, len: 2


In [32]:
columns ={'date': 'date', 'id':'message_id','type':'type', 'from_id': 'from_id', 'actor_id':'actor_id' , 'reply_to_message_id': 'reply_to_message_id',
         'from': 'user_name', 'actor':'user_name_actor'}

In [33]:
df_3 = build_df_chat(result_dict_3, columns)

In [34]:
pd.concat([df_3.head(3),df_3.tail(3)])

,date,chat_name,chat_id,sender_id,sender_type,message_id,type,reply_to_message_id,user_name,first_name,last_name,text
0,2025-02-01 14:06:16,MANAGEMENT ALUMNI PRACTICUM,1586396652,1586396652,channel,856,message,None,MANAGEMENT ALUMNI PRACTICUM,MANAGEMENT,ALUMNI,"💥 💥 💥 💥 января 👋 \n\nПока мы вспоминали, ..."
1,2025-02-02 14:07:55,MANAGEMENT ALUMNI PRACTICUM,1586396652,1586396652,channel,857,message,None,MANAGEMENT ALUMNI PRACTICUM,MANAGEMENT,ALUMNI,💭 Переговорный клуб \n\n→ 6 февраля в 20:00 ...
2,2025-02-03 14:04:43,MANAGEMENT ALUMNI PRACTICUM,1586396652,1586396652,channel,858,message,None,MANAGEMENT ALUMNI PRACTICUM,MANAGEMENT,ALUMNI,У Практикума новый знак ➡️ у выпускников нов...
10,2025-02-20 13:59:58,MANAGEMENT ALUMNI PRACTICUM,1586396652,1586396652,channel,866,message,None,MANAGEMENT ALUMNI PRACTICUM,MANAGEMENT,ALUMNI,Ваш ID выпускника — и скидки по нему \n\nТаког...
11,2025-02-21 15:08:07,MANAGEMENT ALUMNI PRACTICUM,1586396652,1586396652,channel,867,message,None,MANAGEMENT ALUMNI PRACTICUM,MANAGEMENT,ALUMNI,Вебинар «Проблема власти в среде руководителей...
12,2025-02-26 11:59:58,MANAGEMENT ALUMNI PRACTICUM,1586396652,1586396652,channel,868,message,None,MANAGEMENT ALUMNI PRACTICUM,MANAGEMENT,ALUMNI,Вебинар «Проблема власти в среде руководителей...


### 4.  Итоговый  скрипт

In [35]:
path_list

[WindowsPath('C:/Users/dmitr/project_loc/MainTopicsExtractor/main_topics_cloud/reserch/data/example/result.json'),
 WindowsPath('C:/Users/dmitr/project_loc/MainTopicsExtractor/main_topics_cloud/reserch/data/дата/result.json'),
 WindowsPath('C:/Users/dmitr/project_loc/MainTopicsExtractor/main_topics_cloud/reserch/data/маркетинг/result.json'),
 WindowsPath('C:/Users/dmitr/project_loc/MainTopicsExtractor/main_topics_cloud/reserch/data/менеджмент/result.json')]

In [36]:
columns ={'date': 'date', 'id':'message_id','type':'type', 'from_id': 'from_id', 'actor_id':'actor_id' , 'reply_to_message_id': 'reply_to_message_id',
         'from': 'user_name', 'actor':'user_name_actor'}

In [37]:
class DictToDataFrameParser:
    '''
    Атрибуты:
    path (Path): Путь к JSON-файлу, из которого будет загружен словарь.
    columns (dict): Словарь, где ключи соответствуют ключам из JSON-структуры, 
                    а значения — названиям столбцов в первичном DataFrame.
    result_dict (dict): Десериализованный словарь, полученный из JSON-файла.
    df (DataFrame): DataFrame, созданный на основе данных из result_dict и columns

    Методы:
    load_json(path): Загружает и десериализует JSON-файл.(статистический)
    build_df_chat(dict_to_df, columns): Строит DataFrame из десериализованного словаря, 
                                         используя заданные столбцы.(статистический)
    save_to_csv(file_path): Сохраняет DataFrame в CSV-файл по заданному пути.
    '''
    def __init__(self, path: Path, columns: dict[str, str]):
        '''
        :param path: Путь к JSON-файлу для загрузки данных.
        :param columns: Словарь сопоставления ключей из 'messages' с названиями столбцов для первичного DataFrame.
        '''
        self.path = path
        self.columns = columns
        self.result_dict = DictToDataFrameParser.load_json(path)  # Здесь вызываем метод через имя класса
        self.df = DictToDataFrameParser.build_df_chat(self.result_dict, self.columns)
        
    @staticmethod
    def load_json(path: Path) -> dict:
         with open(path, 'r', encoding='utf-8') as file:
            return json.load(file)

    #  создает  дата фрейм из словаря
    @staticmethod
    def build_df_chat( dict_to_df : dict, columns: dict[str,str]) -> pd.DataFrame:
        df_dict = {
        **{columns.get(col): [dict_to_df['messages'][x].get(col) for x in range(len(dict_to_df['messages']))]
            for col in columns
        },
        'text': [ ' '.join( i.get('text') for  i in dict_to_df['messages'][x]['text_entities'])
            for x in range( len(dict_to_df['messages']))
        ]}    
        df = pd.DataFrame(df_dict)
        df['date'] =  pd.to_datetime(df['date'])
        if 'user_name_actor' in df.columns:
            df['user_name'] = df['user_name'].fillna(df['user_name_actor'])
            del df['user_name_actor']

        df['chat_name'] =dict_to_df.get('name')
        df['chat_id'] =dict_to_df.get('id')
        column =list(df.columns)
        df = df[column[:1]+ column[-2:]+column[1:-2]]

        if 'user_name' in df.columns:
            df['first_name'] = df['user_name'].apply( lambda x:  x.split()[0] if len( x.split())>0 else '')
            df['last_name'] = df['user_name'].apply( lambda x: x.split()[1] if len( x.split())>1 else '' )
            column =list(df.columns)
            df = df[column[:-3]+column[-2:]+[column[-3]]]

        if 'actor_id' in df.columns:
            df['from_id'] = df['from_id'].fillna(df['actor_id'])
            del df['actor_id']
            # del df['user_name_actor']

        if 'from_id' in df.columns:
            df['sender_id'] = df['from_id'].str.extract(r'(\d+)')
            df['sender_type'] = df['from_id'].str.replace(r'\d+', '', regex=True)
            del df['from_id']
            column =list(df.columns)
            df=df[column[:3]+column[-2:]+column[3:-2]]
        
         # df = df.fillna(0)
        return df

    def save_to_csv(self, name_file: str, path: Path = Path.cwd()) -> None:
        '''
        Сохраняет DataFrame в CSV файл по заданному пути.
        :param name_file: Имя файла (без расширения).
        :param path: Путь, по которому будет сохранён файл. По умолчанию используется текущая рабочая директория.
        
        '''
        path_file = (path/name_file).with_suffix('.csv')
        self.df.to_csv(path_file, index=False)


In [38]:
marketing = DictToDataFrameParser(path_list[1],columns)

In [39]:
marketing.path

WindowsPath('C:/Users/dmitr/project_loc/MainTopicsExtractor/main_topics_cloud/reserch/data/дата/result.json')

In [40]:
marketing.df.head()

,date,chat_name,chat_id,sender_id,sender_type,message_id,type,reply_to_message_id,user_name,first_name,last_name,text
0,2025-02-01 22:40:19,💬 Data Practicum Chat,1379846874,200103497,user,266690,service,NaN,Елизавета,Елизавета,,
1,2025-02-02 21:59:34,💬 Data Practicum Chat,1379846874,217302209,user,266691,service,NaN,Agamet Agametov,Agamet,Agametov,
2,2025-02-03 11:28:38,💬 Data Practicum Chat,1379846874,312724902,user,266693,message,NaN,Olga Varavina,Olga,Varavina,Всем большой привет! Приглашаю на свой уютный ...
3,2025-02-03 11:52:20,💬 Data Practicum Chat,1379846874,1349934990,user,266694,message,266689.0,Илья,Илья,,А у тебя когда будет свой канал про аналитику?
4,2025-02-03 11:52:37,💬 Data Practicum Chat,1379846874,1349934990,user,266695,message,NaN,Илья,Илья,,Будешь туда голосовухи пятиминутные постить
